# train_hyperasの確認

In [1]:
import numpy as np
import pandas as pd
#最大表示列数の指定
pd.set_option('display.max_columns', 300)

%matplotlib inline 
import matplotlib.pyplot as plt

plt.style.use('ggplot') 

In [2]:
race_df = pd.read_csv("../collect_data/csv/cleaned_race_data.csv", sep=",")
horse_df = pd.read_csv("../collect_data/csv/cleaned_horse_data.csv", sep=",")

/var/folders/cr/c0bln9j166v8pyh2cr3g51lm0000gr/T/ipykernel_21454/2104096674.py:2: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  horse_df = pd.read_csv("../collect_data/csv/cleaned_horse_data.csv", sep=",")


In [3]:
#predict_result = pd.read_csv("./predict/train_simple_predicted_test.csv", sep=",")
predict_result = pd.read_csv("./predict/data/train_hyperas_predicted_test.csv", sep=",")

FileNotFoundError: [Errno 2] No such file or directory: './predict/data/train_hyperas_predicted_test.csv'

In [ ]:
# 障害レースは取り除く
predict_result = predict_result[predict_result["is_obstacle"]==0]

: 

In [ ]:
predict_result.head()

: 

In [ ]:
# 予測値を100倍して見やすくする
predict_result["predict_is_tansyo"] = predict_result["predict_is_tansyo"]*100
predict_result["predict_is_hukusyo"] = predict_result["predict_is_hukusyo"]*100

: 

In [ ]:
predict_result.shape

: 

## accuracy の計算
groupbyして、レースごとに確率の最も高い馬を見つける。
正解していたかを判定

In [ ]:
# group by したデータフレームに対して行う処理
def check_is_correct_predict_is_tansyo(one_race_data):
    return one_race_data.loc[one_race_data['predict_is_tansyo'].idxmax()]['is_tansyo']
def check_is_correct_predict_is_hukusyo(one_race_data):
    return one_race_data.loc[one_race_data['predict_is_hukusyo'].idxmax()]['is_hukusyo']

: 

In [ ]:
part_predict_result = predict_result[["race_id","date","horse_number","predict_is_hukusyo", "is_hukusyo","predict_is_tansyo", "is_tansyo"]]

is_tansyo = part_predict_result.groupby('race_id').apply(check_is_correct_predict_is_tansyo)
is_hukusyo = part_predict_result.groupby('race_id').apply(check_is_correct_predict_is_hukusyo)
print("tansyo accuracy: {}".format(is_tansyo.sum()/is_tansyo.size))
print("hukusyo accuracy: {}".format(is_hukusyo.sum()/is_hukusyo.size))

: 

# blending
お互いの欠点を補い合えるかも...？

In [ ]:
blend_df = predict_result.copy()
blend_df["blend"] = 0.5*blend_df["predict_is_tansyo"] + 0.5*blend_df["predict_is_hukusyo"]
blend_df = blend_df[["race_id","date","horse_number", "blend","is_hukusyo", "is_tansyo"]]

def blend_check_is_correct_predict_is_tansyo(one_race_data):
    return one_race_data.loc[one_race_data['blend'].idxmax()]['is_tansyo']
def blend_check_is_correct_predict_is_hukusyo(one_race_data):
    return one_race_data.loc[one_race_data['blend'].idxmax()]['is_hukusyo']

is_tansyo = blend_df.groupby('race_id').apply(blend_check_is_correct_predict_is_tansyo)
is_hukusyo = blend_df.groupby('race_id').apply(blend_check_is_correct_predict_is_hukusyo)
print("tansyo accuracy: {}".format(is_tansyo.sum()/is_tansyo.size))
print("hukusyo accuracy: {}".format(is_hukusyo.sum()/is_hukusyo.size))

: 

## 同着の試合があるかもしれないので取り除く
同着はデータがおかしくなっているので取り除く。
また、10頭以下のレースは取り除く

In [ ]:
merged_df = pd.merge(blend_df, race_df, on="race_id")
merged_df = pd.merge(merged_df, horse_df, on=['race_id','horse_number'])
merged_df['tansyo_return'] = merged_df['is_tansyo']*merged_df['tansyo'] - 100
merged_df["hukusyo_return"] = (merged_df["rank"]==1)*merged_df["hukusyo_first"] + (merged_df["rank"]==2)*merged_df["hukusyo_second"] + (merged_df["rank"]==3)*merged_df["hukusyo_third"]  - 100

: 

In [ ]:
def remove_same(one_horse_data):
    #  数レース分の情報を結合
    result_df = one_horse_data.copy()
    
    if result_df["rank"].duplicated().sum() or result_df["total_horse_number"].iloc[0]<10:
        return 
    else:
        return result_df

: 

In [ ]:
print(merged_df.shape)
removed = merged_df.groupby("race_id").apply(remove_same)
print(removed.shape)

: 

## 可視化

In [ ]:
removed = removed.reset_index(drop=True)
removed.head()

: 

In [ ]:
# blendの分布についてヒストグラム
removed.plot(kind='hist', y='blend' , bins=10, figsize=(16,4), alpha=0.5)

: 

In [ ]:
# オッズと予測値
removed.plot(kind='scatter', x='blend', y='odds')

: 

In [ ]:
# 実際に1位になったものについてのオッズと予測値
removed[removed["rank"]==1].plot(kind='scatter', x='blend', y='odds')

: 

In [ ]:
# 実際に複勝圏内になったものについてのオッズと予測値
removed[removed["rank"]<=3].plot(kind='scatter', x='blend', y='odds')

: 

: 

## 的中率の計算(実際に1位や３位以内であった確率）

In [ ]:
# cut
removed["blend_cut"] = pd.cut(removed["blend"], 10)

: 

In [ ]:
# tansyo
def right_rate_tansyo(one_cut):
    return one_cut["is_tansyo"].sum()/len(one_cut)
    
removed.groupby("blend_cut").apply(right_rate_tansyo).plot()

: 

In [ ]:
# hukusyo
def right_rate_hukusyo(one_cut):
    return one_cut["is_hukusyo"].sum()/len(one_cut)
    
removed.groupby("blend_cut").apply(right_rate_hukusyo).plot()

: 

## 回収率の計算

In [ ]:
def get_return(one_race):
    tmp = one_race.reset_index()
    return tmp.loc[tmp['blend'].idxmax(),['tansyo_return','hukusyo_return','odds','date','blend']]

race_result = removed.groupby("race_id").apply(get_return)

: 

In [ ]:
race_result = race_result.sort_values("date")

: 

In [ ]:
race_result = race_result.set_index("date")

: 

### 単勝

In [ ]:
# 単純に買い続けたとき
simple = race_result["tansyo_return"].cumsum()
simple.plot(figsize=(12, 6))

: 

In [ ]:
# breandがある程度あるときだけ
simple = race_result.loc[(race_result["blend"]>30), 'tansyo_return'].cumsum()
simple.plot()

: 

In [ ]:
# オッズがある範囲のとき
simple = race_result.loc[((race_result["odds"]>10) & (race_result["odds"]<60) & (race_result["blend"]>30)), 'tansyo_return'].cumsum()
simple.plot()

: 

### 複勝

In [ ]:
# 単純に買い続けたとき
simple = race_result["hukusyo_return"].cumsum()
simple.plot(figsize=(12, 6))

: 

In [ ]:
# blendの予測値を含める
simple = race_result.loc[(race_result["blend"]>30), 'hukusyo_return'].cumsum()
simple.plot()

: 

In [ ]:
# オッズがある範囲のとき
simple = race_result.loc[((race_result["odds"]>20.0) & (race_result["odds"]<65)), 'hukusyo_return'].cumsum()
simple.plot(figsize=(12, 6))

: 

: 